# 🛩️ Пример работы с LinearLongitudinalUAV в TensorAeroSpace

Добро пожаловать в интерактивный пример работы с **LinearLongitudinalUAV** - линеаризованной продольной моделью беспилотного летательного аппарата (БПЛА)! ✈️

## 📋 Описание

В этом notebook мы изучим:

- 🔧 **Настройку параметров симуляции** для БПЛА
- 🎯 **Создание опорных сигналов** с помощью unit_step
- 🛩️ **Инициализацию среды** LinearLongitudinalUAV
- 🎮 **Выполнение шагов симуляции** с управляющими воздействиями
- 📊 **Анализ результатов** и состояний системы

LinearLongitudinalUAV представляет собой упрощенную модель продольного движения БПЛА, которая идеально подходит для изучения основ управления летательными аппаратами и разработки алгоритмов автопилота.

---

## 🚀 Настройка среды

Начнем с подготовки рабочего окружения и импорта необходимых библиотек.

### 📦 Импорт библиотек

Импортируем все необходимые модули для работы с БПЛА:

In [1]:
# 📚 Основные библиотеки для работы с окружениями и численными вычислениями
import gymnasium as gym
import numpy as np
from tqdm import tqdm

# 🛩️ Специализированные модули TensorAeroSpace для БПЛА
from tensoraerospace.envs import LinearLongitudinalUAV
from tensoraerospace.utils import generate_time_period, convert_tp_to_sec_tp
from tensoraerospace.signals.standart import unit_step

print("✅ Все библиотеки успешно импортированы!")
print("🛩️ Готовы к работе с LinearLongitudinalUAV")

### ⚙️ Параметры симуляции

Настроим основные параметры для симуляции БПЛА:

In [2]:
# ⏱️ Временные параметры симуляции
dt = 0.01  # Шаг дискретизации (секунды) - определяет точность симуляции
tp = generate_time_period(tn=20, dt=dt)  # Временной период: 20 секунд симуляции
tps = convert_tp_to_sec_tp(tp, dt=dt)  # Преобразование в секунды
number_time_steps = len(tp)  # Общее количество временных шагов

# 🎯 Создание опорного сигнала (целевая траектория)
# unit_step создает ступенчатый сигнал: 5 градусов на 10-м шаге
reference_signals = np.reshape(
    unit_step(degree=5, tp=tp, time_step=10, output_rad=True), 
    [1, -1]
)

# 📊 Вывод информации о параметрах симуляции
print(f"⏱️ Шаг дискретизации: {dt} сек")
print(f"🕐 Общее время симуляции: {tp[-1]:.1f} сек")
print(f"📈 Количество временных шагов: {number_time_steps}")
print(f"🎯 Опорный сигнал: ступенька {np.degrees(reference_signals[0, 15]):.1f}° на {10*dt:.1f} сек")

### 🛩️ Создание среды БПЛА

Инициализируем среду LinearLongitudinalUAV с заданными параметрами:

In [3]:
# 🏗️ Создание среды LinearLongitudinalUAV
env = gym.make(
    'LinearLongitudinalUAV-v0',  # Идентификатор среды БПЛА
    number_time_steps=number_time_steps,  # Количество временных шагов
    initial_state=[[0],[0],[0],[0]],  # Начальное состояние: [u, w, q, θ] = [0, 0, 0, 0]
    output_space=None,  # Автоматическое определение пространства выходов
    reference_signal=reference_signals  # Опорный сигнал для отслеживания
)

# 🔄 Сброс среды в начальное состояние
initial_observation, info = env.reset()

# 📊 Анализ начального состояния БПЛА
print("🛩️ Среда LinearLongitudinalUAV успешно создана!")
print("\n📋 Начальное состояние БПЛА:")
print(f"   u (продольная скорость): {initial_observation[0][0]:.3f} м/с")
print(f"   w (вертикальная скорость): {initial_observation[1][0]:.3f} м/с")
print(f"   q (угловая скорость тангажа): {initial_observation[2][0]:.3f} рад/с")
print(f"   θ (угол тангажа): {np.degrees(initial_observation[3][0]):.3f}°")

initial_observation

/Users/asmazaev/Projects/TensorAeroSpace/.venv/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:159: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


(array([[0.],
        [0.],
        [0.],
        [0.]], dtype=float32),
 {})

### 🎮 Выполнение шага симуляции

Применим управляющее воздействие и выполним один шаг симуляции:

In [5]:
# 🎮 Применение управляющего воздействия
# Подаем управляющий сигнал: 1 единица (например, отклонение руля высоты)
control_input = np.array([[1]])  # Управляющее воздействие

# 🚀 Выполнение одного шага симуляции
observation, reward, terminated, truncated, info = env.step(control_input)

# 📊 Вывод информации о выполненном шаге
print("🎮 Шаг симуляции выполнен!")
print(f"🎯 Управляющее воздействие: {control_input[0][0]}")
print(f"🏁 Эпизод завершен: {terminated}")
print(f"⏰ Время истекло: {truncated}")
print(f"📈 Награда: {reward[0]:.6f}")

### 🔍 Анализ внутренних данных модели

Исследуем историю управляющих воздействий, сохраненную в модели:

In [6]:
# 📚 Получение истории управляющих воздействий
input_history = env.model.store_input

# 🔍 Анализ истории управления
print("📚 История управляющих воздействий:")
print(f"📏 Размерность: {input_history.shape}")
print(f"🎮 Количество ненулевых воздействий: {np.count_nonzero(input_history)}")
print(f"🎯 Первые 5 значений: {input_history[0, :5]}")
print(f"📊 Максимальное воздействие: {np.max(input_history):.3f}")
print(f"📉 Минимальное воздействие: {np.min(input_history):.3f}")

input_history

/Users/asmazaev/Projects/TensorAeroSpace/.venv/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.model to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.model` for environment variables or `env.get_wrapper_attr('model')` that will search the reminding wrappers.
  logger.warn(


array([[1., 1., 0., ..., 0., 0., 0.]])

### 🎯 Анализ опорного сигнала

Исследуем опорный сигнал, который БПЛА должен отслеживать:

In [7]:
# 🎯 Получение опорного сигнала
ref_signal = env.reference_signal
current_ref_value = env.reference_signal[0][1]  # Текущее значение на втором шаге

# 📊 Анализ опорного сигнала
print("🎯 Анализ опорного сигнала:")
print(f"📏 Размерность: {ref_signal.shape}")
print(f"🔢 Текущее значение (шаг 1): {current_ref_value:.3f} рад")
print(f"📐 В градусах: {np.degrees(current_ref_value):.3f}°")
print(f"🎯 Значение после ступеньки (шаг 15): {np.degrees(ref_signal[0, 15]):.3f}°")
print(f"📈 Максимальное значение: {np.degrees(np.max(ref_signal)):.3f}°")
print(f"📉 Минимальное значение: {np.degrees(np.min(ref_signal)):.3f}°")

current_ref_value

/Users/asmazaev/Projects/TensorAeroSpace/.venv/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.reference_signal to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.reference_signal` for environment variables or `env.get_wrapper_attr('reference_signal')` that will search the reminding wrappers.
  logger.warn(


0.0

### 📈 Анализ награды

Исследуем систему наград, которая оценивает качество управления БПЛА:

In [8]:
# 📈 Анализ системы наград
current_reward = reward[0]

# 🔍 Интерпретация награды
print("📈 Анализ награды за текущий шаг:")
print(f"🎯 Значение награды: {current_reward:.6f}")

if current_reward > 0:
    print("✅ Положительная награда - хорошее управление!")
    print("📊 БПЛА движется в правильном направлении")
elif current_reward == 0:
    print("⚖️ Нейтральная награда - стабильное состояние")
else:
    print("❌ Отрицательная награда - требуется коррекция")
    print("🔧 Необходимо улучшить управляющие воздействия")

print(f"\n💡 Интерпретация: Награда {current_reward:.6f} указывает на")
print("   качество отслеживания опорного сигнала БПЛА")

reward

array([0.00463475])

## 🎓 Заключение

### 📚 Что мы изучили:

1. **🚁 Модель БПЛА**: Познакомились с линейной продольной моделью беспилотного летательного аппарата
2. **🎮 Среда Gymnasium**: Научились создавать и взаимодействовать со средой `LinearLongitudinalUAV-v0`
3. **📊 Анализ состояний**: Изучили компоненты состояния БПЛА (скорости, угловые скорости, углы)
4. **🎯 Управление**: Поняли принципы применения управляющих воздействий
5. **📈 Система наград**: Разобрали механизм оценки качества управления
6. **🔍 Мониторинг**: Освоили методы анализа внутренних данных модели

### 🚀 Следующие шаги:

- **🤖 Обучение RL-агентов**: Используйте эту среду для тренировки алгоритмов обучения с подкреплением
- **📐 Настройка параметров**: Экспериментируйте с различными параметрами симуляции
- **🎯 Сложные траектории**: Создавайте более сложные опорные сигналы
- **📊 Визуализация**: Добавьте графики для анализа поведения БПЛА

### 📖 Полезные ресурсы:

- [Документация TensorAeroSpace](https://github.com/TensorAeroSpace/TensorAeroSpace)
- [Gymnasium Documentation](https://gymnasium.farama.org/)
- [Reinforcement Learning: An Introduction](http://incompleteideas.net/book/the-book.html)

---

*💡 Этот notebook демонстрирует основы работы с моделью БПЛА в TensorAeroSpace. Используйте его как отправную точку для ваших исследований в области управления летательными аппаратами!*